## Web Scrape Salary Data of Data Analyst I from Salary.com

In [1]:
import re
import csv
import json
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
import requests

### Get the URLs for All Cities

<font size=4>First, examine https://www.salary.com/research/salary/benchmark/data-analyst-i-salary/juneau-ak. 
<br><br>By observation, only the last text <font color=red>(juneau-ak)</font> of the URLs determines which city we are considering and we need to get this text for all cities for further web scraping.
<br><br>Therefore, inspect https://www.salary.com/research/salary/select-location and scrape the required text.</font>

In [2]:
state_city_url = "https://www.salary.com/research/salary/select-location"
state_city = requests.get(state_city_url)
soup_state_city = BeautifulSoup(state_city.content, 'lxml')

In [3]:
cities = soup_state_city.find_all('div', class_='sal-city-list sal-statecode-wrapper')[1:]
temp0 = []
for city in cities:
    for link in city.find_all('a'):
        temp0.append(link['href'])
city_state_list = [link.split('/')[-1] for link in temp0]

In [4]:
city_state_list[:5]

['anchorage-ak', 'fairbanks-ak', 'juneau-ak', 'ketchikan-ak', 'palmer-ak']

In [5]:
len(city_state_list)

808

### Get the Data for Each City

<font size=4>To begin with, first examine the https://www.salary.com/research/salary/benchmark/data-analyst-i-salary/anchorage-ak which is the URL for Anchorage, Alaska.</font>

In [6]:
template = "https://www.salary.com/research/salary/benchmark/data-analyst-i-salary/"
template1 = template+city_state_list[0]
template1

'https://www.salary.com/research/salary/benchmark/data-analyst-i-salary/anchorage-ak'

<font size=4>The data we want is stored in the `table` tag with `table-chart` class</font>

In [10]:
city = requests.get(template1)
soup_city = BeautifulSoup(city.content, 'lxml')
title = soup_city.find('table', class_='table-chart')
title

<table class="table-chart">
<thead>
<tr>
<td>Percentile</td>
<td>Salary</td>
<td>Location</td>
<td class="nowrap">Last Updated</td>
</tr>
</thead>
<tbody>
<tr>
<td>10th Percentile Data Analyst I Salary</td>
<td>$56,046</td>
<td>Anchorage,AK</td>
<td class="nowrap">June 26, 2023</td>
</tr>
<tr>
<td>25th Percentile Data Analyst I Salary</td>
<td>$64,286</td>
<td>Anchorage,AK</td>
<td class="nowrap">June 26, 2023</td>
</tr>
<tr>
<td>50th Percentile Data Analyst I Salary</td>
<td>$73,336</td>
<td>Anchorage,AK</td>
<td class="nowrap">June 26, 2023</td>
</tr>
<tr>
<td>75th Percentile Data Analyst I Salary</td>
<td>$83,762</td>
<td>Anchorage,AK</td>
<td class="nowrap">June 26, 2023</td>
</tr>
<tr>
<td>90th Percentile Data Analyst I Salary</td>
<td>$93,253</td>
<td>Anchorage,AK</td>
<td class="nowrap">June 26, 2023</td>
</tr>
</tbody>
</table>

<font size=4>To extract the data, we need to further find the `tr` tag and `td` tag as follows:</font>

In [11]:
column_data = title.find_all('tr')
column_data

[<tr>
 <td>Percentile</td>
 <td>Salary</td>
 <td>Location</td>
 <td class="nowrap">Last Updated</td>
 </tr>,
 <tr>
 <td>10th Percentile Data Analyst I Salary</td>
 <td>$56,046</td>
 <td>Anchorage,AK</td>
 <td class="nowrap">June 26, 2023</td>
 </tr>,
 <tr>
 <td>25th Percentile Data Analyst I Salary</td>
 <td>$64,286</td>
 <td>Anchorage,AK</td>
 <td class="nowrap">June 26, 2023</td>
 </tr>,
 <tr>
 <td>50th Percentile Data Analyst I Salary</td>
 <td>$73,336</td>
 <td>Anchorage,AK</td>
 <td class="nowrap">June 26, 2023</td>
 </tr>,
 <tr>
 <td>75th Percentile Data Analyst I Salary</td>
 <td>$83,762</td>
 <td>Anchorage,AK</td>
 <td class="nowrap">June 26, 2023</td>
 </tr>,
 <tr>
 <td>90th Percentile Data Analyst I Salary</td>
 <td>$93,253</td>
 <td>Anchorage,AK</td>
 <td class="nowrap">June 26, 2023</td>
 </tr>]

In [12]:
column_data[1].find_all('td')

[<td>10th Percentile Data Analyst I Salary</td>,
 <td>$56,046</td>,
 <td>Anchorage,AK</td>,
 <td class="nowrap">June 26, 2023</td>]

<font size=4>Generalize above logic and put them into a function and use `try` and `except` to deal with connection error problems.</font>

In [26]:
def one_row(city_state):
    template = "https://www.salary.com/research/salary/benchmark/data-analyst-i-salary/"
    template1 = template+city_state
    try:
        city = requests.get(template1)
        if city.status_code != 200:
            return ['NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA']
    except requests.exceptions.ConnectionError:
        return ['NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA']
    soup_city = BeautifulSoup(city.content, 'lxml')
    title = soup_city.find('table', class_='table-chart')
    column_data = title.find_all('tr')
    temp_table=[]
    for data in column_data[1:]:
        # store the 10, 25, 50, 75, 90
        temp_table.append(int(data.find_all('td')[1].text.strip('$').replace(',', '')))
    # Store city, State
    temp_final_table = column_data[5].find_all('td')[2].text.split(',') + temp_table
    return temp_final_table

<font size=4>Build a new dataframe for storing the data. For saving time, only the first 200 city in the `city_state_list`.</font>

In [ ]:
df = pd.DataFrame(columns=['city', 'state', '10th', '25th', '50th', '75th', '90th'])
for code in city_state_list[:200]:
    length = len(df)
    df.loc[length] = one_row(code)
    print(df.loc[length])
    sleep(0.5)

In [32]:
df.head()

,city,state,10th,25th,50th,75th,90th
0,Anchorage,AK,56046,64286,73336,83762,93253
1,Fairbanks,AK,55814,64020,73033,83415,92868
2,Juneau,AK,55547,63713,72683,83016,92423
3,Ketchikan,AK,54791,62846,71694,81886,91166
4,Palmer,AK,55290,63419,72347,82632,91996


In [37]:
df.tail()

,city,state,10th,25th,50th,75th,90th
195,Port Saint Lucie,FL,47510,54495,62168,71005,79052
196,Saint Petersburg,FL,47973,55026,62773,71697,79821
197,Sarasota,FL,48124,55199,62971,71923,80073
198,Spring Hill,FL,47574,54568,62251,71100,79157
199,Tallahassee,FL,45402,52078,59409,67855,75544


In [39]:
df.to_csv('salary.csv', index=False)